In [ ]:
# importar libreria
import pandas as pd
import numpy as np
from  google.colab import files
import xlrd
import re

In [ ]:
ANNO=int(input("Insertar año: "))
#mes=int(input("Insertar mes en su valor numerico (ejemplo: Enero = 1): "))
regimen=int(input("Insertar el regimen de la tabla(Eje: CAS=1): "))
mesInicio=int(input("Se va a insertar un intervalo de meses a filtrar, si solo quiere filtrar un mes entonces repetir el numero del mes \nInsertar el primer mes en su valor numerico (ejemplo: Enero = 1): "))
mesFin=int(input("Insertar el ultimo mes en su valor numerico (ejemplo: Diciembre = 12, mes maximo): "))

nomExcel = str(input("INSERTAR EL NOMBRE DEL ARCHIVO FINAL A DESCARGAR (ejemplo: a123.xlsx)"))

# Cargar datos
direccionLocal=str(input("INSERTAR LA RUTA DEL ARCHIVO (tabla) A SER PROCESADO (ejemplo: /content/A.xlsx)"))

while (mesInicio<=mesFin<=12):
  mes=mesInicio


  if mes==1:
    nomMes='ENERO'
    print("El mes insertado es: "+nomMes)
  elif mes==2:
    nomMes='FEBRERO'
    print("El mes insertado es: "+nomMes)
  elif mes==3:
    nomMes='MARZO'
    print("El mes insertado es: "+nomMes)
  elif mes==4:
    nomMes='ABRIL'
    print("El mes insertado es: "+nomMes)
  elif mes==5:
    nomMes='MAYO'
    print("El mes insertado es: "+nomMes)
  elif mes==6:
    nomMes='JUNIO'
    print("El mes insertado es: "+nomMes)
  elif mes==7:
    nomMes='JULIO'
    print("El mes insertado es: "+nomMes)
  elif mes==8:
    nomMes='AGOSTO'
    print("El mes insertado es: "+nomMes)
  elif mes==9:
    nomMes='SEPTIEMBRE'
    print("El mes insertado es: "+nomMes)
  elif mes==10:
    nomMes='OCTUBRE'
    print("El mes insertado es: "+nomMes)
  elif mes==11:
    nomMes='NOVIEMBRE'
    print("El mes insertado es: "+nomMes)
  elif mes==12:
    nomMes='DICIEMBRE'
    print("El mes insertado es: "+nomMes)
  else:
    print("Error en el mes insertado: Se sugiere insertar otro Mes")


  #direccionLocal="/content/TABLAS DATA/RÉGIMEN DL.728 - CONTRATADOS A PLAZO DETERMINADO.xlsx"
  df = pd.read_excel(direccionLocal, index_col= 0)


  filtroFebrero = df[(df[nomMes].notna()) & (df[nomMes] != 0) & (df[nomMes] != '-')]

  #columna de los digitos que tiene el DNI
  filtroFebrero['DIGITOS_DNI']=filtroFebrero['DNI'].apply(lambda x: len(str(x)))

  #columnas iniciales
  filtroFebrero["'VC_PERSONAL_RUC_ENTIDAD"]=np.where(filtroFebrero[nomMes]!='[]', 20162290089, 'ERROR')
  filtroFebrero["'VC_PERSONAL_RUC_ENTIDAD"]=filtroFebrero["'VC_PERSONAL_RUC_ENTIDAD"].astype(int)
  filtroFebrero["'IN_PERSONAL_ANNO"]=np.where(filtroFebrero[nomMes]!='[]', ANNO, 'ERROR')
  filtroFebrero["'IN_PERSONAL_ANNO"]=filtroFebrero["'IN_PERSONAL_ANNO"].astype(int)
  filtroFebrero["'IN_PERSONAL_MES"]=np.where(filtroFebrero[nomMes]!='[]', mes, 'ERROR')


  def dni(x):
    if(str(x)[0:1]=="´"):
      dni=x[1:]
      return dni
    else:
      dni='0'+str(x)[0:]
      return dni


  filtroFebrero["'VC_PERSONAL_DNI"]=np.where(filtroFebrero['DIGITOS_DNI'] == 8, filtroFebrero['DNI'], filtroFebrero['DNI'].apply(lambda x : dni(x)))

  filtroFebrero["'VC_PERSONAL_REGIMEN_LABORAL"]=np.where(filtroFebrero[nomMes]!='[]', regimen, 'ERROR')


  #Aplicar algritmo para obtener el apellido paterno y materno
  def apellidoPaterno(x):
    if (x.find('DE LA ')==0):
      apellidoNuevo=x[6:]
      posFin=apellidoNuevo.find(' ')
      apellidoPaterno='DE LA '+apellidoNuevo[0:posFin]
      return apellidoPaterno

    elif (x.find('DE ')==0):
      apellidoNuevo=x[3:]
      posFin=apellidoNuevo.find(' ')
      apellidoPaterno='DE '+apellidoNuevo[0:posFin]
      return apellidoPaterno

    elif (x.find('LA ')==0):
      apellidoNuevo=x[3:]
      posFin=apellidoNuevo.find(' ')
      apellidoPaterno='LA '+apellidoNuevo[0:posFin]
      return apellidoPaterno

    else:
      posFin=x.find(' ')
      apellidoPaterno=x[0:posFin]
      return apellidoPaterno

  def apellidoMaterno(x):
    if (x.find('DE LA ')==0):
      apellidoNuevo=x[6:]
      posFin=apellidoNuevo.find(' ')+1
      apellidoMaterno=apellidoNuevo[posFin:]
      return apellidoMaterno

    elif (x.find('DE ')==0):
      apellidoNuevo=x[3:]
      posFin=apellidoNuevo.find(' ')+1
      apellidoMaterno=apellidoNuevo[posFin:]
      return apellidoMaterno

    elif (x.find('LA ')==0):
      apellidoNuevo=x[3:]
      posFin=apellidoNuevo.find(' ')+1
      apellidoMaterno=apellidoNuevo[posFin:]
      return apellidoMaterno

    else:
      posFin=x.find(' ')+1
      apellidoMaterno=x[posFin:]
      return apellidoMaterno

  filtroFebrero["'VC_PERSONAL_PATERNO"]=filtroFebrero["APELLIDOS"].apply(lambda x: apellidoPaterno(x))
  filtroFebrero["'VC_PERSONAL_MATERNO"]=filtroFebrero["APELLIDOS"].apply(lambda x: apellidoMaterno(x))
  #codigo inicial
  #filtroFebrero["'VC_PERSONAL_PATERNO"]=np.where(filtroFebrero[nomMes]!='[]', filtroFebrero['APELLIDOS'], 'ERROR')#ver que codigo va
  #filtroFebrero["'VC_PERSONAL_MATERNO"]=np.where(filtroFebrero[nomMes]!='[]', '', 'ERROR')#ver que codigo va
  filtroFebrero["'VC_PERSONAL_NOMBRES"]=np.where(filtroFebrero[nomMes]!='[]', filtroFebrero['NOMBRES'], 'ERROR')


  filtroFebrero["'VC_PERSONAL_CARGO"]=np.where(filtroFebrero[nomMes]!='[]', filtroFebrero['CARGO'], 'ERROR')
  filtroFebrero["'VC_PERSONAL_DEPENDENCIA"]=np.where(filtroFebrero[nomMes]!='[]', filtroFebrero['DEPENDENCIA'], 'ERROR')
  filtroFebrero["'MO_PERSONAL_REMUNERACIONES"]=np.where(filtroFebrero[nomMes]!='[]', filtroFebrero[nomMes], 'ERROR')

  #AQUI VAN LOS EXTRAS
  filtroFebrero["'MO_PERSONAL_HONORARIOS"]=np.where(filtroFebrero[nomMes]!='[]', 0, 'ERROR')
  filtroFebrero["'MO_PERSONAL_INCENTIVO"]=np.where(filtroFebrero[nomMes]!='[]', 0, 'ERROR')


  #AQUI EL AGUINALDO O GRATIFICACION columna: 'MO_PERSONAL_GRATIFICACION
  if(mes==1):

    filtroFebrero["'MO_PERSONAL_GRATIFICACION"]=np.where((filtroFebrero['ESCOLARIDAD ENERO 2020'].notna()) & (filtroFebrero['ESCOLARIDAD ENERO 2020']!=0), filtroFebrero['ESCOLARIDAD ENERO 2020'], 0)
  elif mes==7:
    filtroFebrero["'MO_PERSONAL_GRATIFICACION"]=np.where((filtroFebrero['AGUINALDO JULIO'].notna()) & (filtroFebrero['AGUINALDO JULIO']!= 0) & (filtroFebrero['AGUINALDO JULIO']!='-'), filtroFebrero['AGUINALDO JULIO'], 0)
    filtroFebrero["'MO_PERSONAL_GRATIFICACION"]=np.where((filtroFebrero["'MO_PERSONAL_GRATIFICACION"]==0) & (filtroFebrero['GRATIFICACION JULIO'].notna()) & (filtroFebrero['GRATIFICACION JULIO']!= 0) & (filtroFebrero['GRATIFICACION JULIO']!='-'), filtroFebrero['GRATIFICACION JULIO'], filtroFebrero['AGUINALDO JULIO'])
    filtroFebrero["'MO_PERSONAL_GRATIFICACION"]=np.where((filtroFebrero["'MO_PERSONAL_GRATIFICACION"].notna()) & (filtroFebrero["'MO_PERSONAL_GRATIFICACION"]!='-'), filtroFebrero["'MO_PERSONAL_GRATIFICACION"], 0)
  elif mes==12:
    filtroFebrero["'MO_PERSONAL_GRATIFICACION"]=np.where((filtroFebrero['AGUINALDO DICIEMBRE'].notna()) & (filtroFebrero['AGUINALDO DICIEMBRE']!= 0) & (filtroFebrero['AGUINALDO DICIEMBRE']!= '-'), filtroFebrero['AGUINALDO DICIEMBRE'], 0)
    filtroFebrero["'MO_PERSONAL_GRATIFICACION"]=np.where((filtroFebrero["'MO_PERSONAL_GRATIFICACION"]==0) & (filtroFebrero['GRATIFICACION DICIEMBRE'].notna()) & (filtroFebrero['GRATIFICACION DICIEMBRE']!= 0) & (filtroFebrero['GRATIFICACION DICIEMBRE']!= '-'), filtroFebrero['GRATIFICACION DICIEMBRE'], filtroFebrero["'MO_PERSONAL_GRATIFICACION"])
    filtroFebrero["'MO_PERSONAL_GRATIFICACION"]=np.where((filtroFebrero["'MO_PERSONAL_GRATIFICACION"].notna()) & (filtroFebrero["'MO_PERSONAL_GRATIFICACION"]!='-'), filtroFebrero["'MO_PERSONAL_GRATIFICACION"], 0)

  else:
    filtroFebrero["'MO_PERSONAL_GRATIFICACION"]=np.where(filtroFebrero[nomMes]!='[]', 0, 'ERROR')


  #AQUI OTROS BENEFICIOS
  filtroFebrero["'MO_PERSONAL_OTROS_BENEFICIOS"]=np.where(filtroFebrero[nomMes]!='[]', 0, 'ERROR')
  #AQUI EL TOTAL
  def convertirCadenaFloat(x):
    if(x.find(',')!=-1):
      posFin=x.find(',')
      posIni=x.find(',')+1
      numPunto=x[0:posFin]+'.'+str(x)[posIni:]
      num=float(numPunto)
      return num
    elif(x.find('.')!=-1):
      num=float(x)
      return num
    else:
      num=float(x)
      return num

  sueldoTotal=filtroFebrero[["'MO_PERSONAL_REMUNERACIONES", "'MO_PERSONAL_GRATIFICACION"]].astype(str)
  sueldoTotal["'MO_PERSONAL_REMUNERACIONES"]=np.where(filtroFebrero[nomMes]!='[]', sueldoTotal["'MO_PERSONAL_REMUNERACIONES"].apply(lambda x: convertirCadenaFloat(x)),'ERROR')
  sueldoTotal["'MO_PERSONAL_GRATIFICACION"]=np.where(filtroFebrero[nomMes]!='[]', sueldoTotal["'MO_PERSONAL_GRATIFICACION"].apply(lambda x: convertirCadenaFloat(x)), 'ERROR')
  sueldoTotal=sueldoTotal[["'MO_PERSONAL_REMUNERACIONES", "'MO_PERSONAL_GRATIFICACION"]].astype(float)
  sueldoTotal["'MO_PERSONAL_TOTAL"]=sueldoTotal.sum(axis=1)
  filtroFebrero["'MO_PERSONAL_GRATIFICACION"]=sueldoTotal[["'MO_PERSONAL_GRATIFICACION"]]
  filtroFebrero["'MO_PERSONAL_TOTAL"]=sueldoTotal[["'MO_PERSONAL_TOTAL"]]

  #OBSERVACIONES
  filtroFebrero["'VC_PERSONAL_OBSERVACIONES"]=np.where(filtroFebrero[nomMes]!='[]', '', 'ERROR')
  #filtroFebrero[["'VC_PERSONAL_OBSERVACIONES", "'MO_PERSONAL_REMUNERACIONES", "'MO_PERSONAL_HONORARIOS", "'MO_PERSONAL_INCENTIVO", "'MO_PERSONAL_OTROS_BENEFICIOS"]].astype(float)

  #transformando tipos de datos
  filtroFebrero["'IN_PERSONAL_MES"]=filtroFebrero["'IN_PERSONAL_MES"].astype(int)
  filtroFebrero["'VC_PERSONAL_REGIMEN_LABORAL"]=filtroFebrero["'VC_PERSONAL_REGIMEN_LABORAL"].astype(int)
  filtroFebrero["'MO_PERSONAL_HONORARIOS"]=filtroFebrero["'MO_PERSONAL_HONORARIOS"].astype(int)
  filtroFebrero["'MO_PERSONAL_INCENTIVO"]=filtroFebrero["'MO_PERSONAL_INCENTIVO"].astype(int)
  filtroFebrero["'MO_PERSONAL_OTROS_BENEFICIOS"]=filtroFebrero["'MO_PERSONAL_OTROS_BENEFICIOS"].astype(int)
  filtroFebrero["'MO_PERSONAL_REMUNERACIONES"]=filtroFebrero["'MO_PERSONAL_REMUNERACIONES"].astype(float)


  #tablaTodo=filtroFebrero.loc
  filtroTotal=filtroFebrero.loc[:, "'VC_PERSONAL_RUC_ENTIDAD":"'VC_PERSONAL_OBSERVACIONES"]


  nameExcel= nomExcel + ' ' + nomMes + ' ' +str(ANNO) +'.xlsx'
  filtroTotal.to_excel(nameExcel)
  files.download(nameExcel)

  #reinicio del bucle
  mesInicio=mesInicio+1
  nameExcel=''


Insertar año: 3042
Insertar el regimen de la tabla(Eje: CAS=1): 1
Se va a insertar un intervalo de meses a filtrar, si solo quiere filtrar un mes entonces repetir el numero del mes 
Insertar el primer mes en su valor numerico (ejemplo: Enero = 1): 1
Insertar el ultimo mes en su valor numerico (ejemplo: Diciembre = 12, mes maximo): 1
INSERTAR EL NOMBRE DEL ARCHIVO FINAL A DESCARGAR (ejemplo: a123.xlsx)l.xlsx
INSERTAR LA RUTA DEL ARCHIVO (tabla) A SER PROCESADO (ejemplo: /content/A.xlsx)/content/CAS 2022.xlsx
El mes insertado es: ENERO


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>